In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'pycaret'

In [3]:
train = pd.read_csv('airline_dataset/train.csv')
train = train.drop('id', axis=1)
test = pd.read_csv('airline_dataset/train.csv')
test = test.drop('id', axis=1)

train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,Female,disloyal Customer,22,Business travel,Eco,1599,3,0,3,3,...,4,5,4,4,4,5,4,0,0.0,0
1,Female,Loyal Customer,37,Business travel,Business,2810,2,4,4,4,...,5,5,4,2,1,5,2,18,18.0,0
2,Male,Loyal Customer,46,Business travel,Business,2622,1,1,1,1,...,4,4,4,4,5,4,3,0,0.0,1
3,Female,disloyal Customer,24,Business travel,Eco,2348,3,3,3,3,...,3,2,4,5,3,4,3,10,2.0,0
4,Female,Loyal Customer,58,Business travel,Business,105,3,3,3,3,...,4,4,4,4,4,4,5,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Male,Loyal Customer,30,Personal Travel,Eco,2377,1,5,1,1,...,1,3,5,3,4,2,4,211,225.0,0
2996,Female,disloyal Customer,24,Business travel,Eco,1643,2,4,3,4,...,5,2,2,4,1,3,5,20,13.0,0
2997,Female,disloyal Customer,22,Business travel,Eco,1442,2,2,2,3,...,4,3,2,3,4,3,4,64,67.0,0
2998,Female,disloyal Customer,33,Business travel,Business,2158,2,2,2,5,...,4,5,2,5,5,5,4,0,3.0,0


In [84]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             3000 non-null   object 
 1   Customer Type                      3000 non-null   object 
 2   Age                                3000 non-null   int64  
 3   Type of Travel                     3000 non-null   object 
 4   Class                              3000 non-null   object 
 5   Flight Distance                    3000 non-null   int64  
 6   Seat comfort                       3000 non-null   int64  
 7   Departure/Arrival time convenient  3000 non-null   int64  
 8   Food and drink                     3000 non-null   int64  
 9   Gate location                      3000 non-null   int64  
 10  Inflight wifi service              3000 non-null   int64  
 11  Inflight entertainment             3000 non-null   int64

In [79]:
continuous_feature = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
ordinal_feature = ['Seat comfort', 'Departure/Arrival time convenient', 'Food and drink', 'Gate location', 
           'Inflight wifi service', 'Inflight entertainment', 'Online support', 'Ease of Online booking', 
           'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding']
categorical_feature = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

In [80]:
for f in categorical_feature:
    print(train[f].value_counts())

Male      1502
Female    1498
Name: Gender, dtype: int64
Loyal Customer       2489
disloyal Customer     511
Name: Customer Type, dtype: int64
Business travel    2053
Personal Travel     947
Name: Type of Travel, dtype: int64
Business    1434
Eco         1337
Eco Plus     229
Name: Class, dtype: int64


In [92]:
enc = OrdinalEncoder()

train[categorical_feature] = enc.fit_transform(train[categorical_feature])
train[categorical_feature] = train[categorical_feature].astype(int)
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,0,1,22,0,1,1599,3,0,3,3,...,4,5,4,4,4,5,4,0,0.0,0
1,0,0,37,0,0,2810,2,4,4,4,...,5,5,4,2,1,5,2,18,18.0,0
2,1,0,46,0,0,2622,1,1,1,1,...,4,4,4,4,5,4,3,0,0.0,1
3,0,1,24,0,1,2348,3,3,3,3,...,3,2,4,5,3,4,3,10,2.0,0
4,0,0,58,0,0,105,3,3,3,3,...,4,4,4,4,4,4,5,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,0,30,1,1,2377,1,5,1,1,...,1,3,5,3,4,2,4,211,225.0,0
2996,0,1,24,0,1,1643,2,4,3,4,...,5,2,2,4,1,3,5,20,13.0,0
2997,0,1,22,0,1,1442,2,2,2,3,...,4,3,2,3,4,3,4,64,67.0,0
2998,0,1,33,0,0,2158,2,2,2,5,...,4,5,2,5,5,5,4,0,3.0,0


## 특성 스케일링

## 모델학습

In [93]:
X = train.drop('target',axis=1)
y = train['target']

In [94]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [95]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [101]:
proba = rf.predict_proba(X_val)
prediction = rf.predict(X_val)

In [100]:
accuracy_score(y_val, prediction)

0.9416666666666667